In [6]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
import time
from ta.trend import macd, macd_diff, macd_signal, ema, ema_indicator
import numpy as np
from ta.momentum import stoch, stoch_signal

In [13]:
sorted_stock = SortedStocksList.objects.get(created_at__date=datetime.now().date(), symbol__symbol="ibventures")
df = sorted_stock.symbol.get_stock_live_data()
today_date = datetime.today().date()
df["macd"] = macd(df.close_price)
df["macd_signal"] = macd_signal(df.close_price)
df["macd_diff"] = macd_diff(df.close_price)
df["percentage"] = round(df.macd * df.macd_diff /100, 6)
df["signal"] = np.where(df.macd < df.macd_signal, "SELL", "BUY")
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
new_df = df.copy(deep=True).drop(df.tail(1).index)
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = new_df[new_df.signal.str.endswith("SELL_CROSSOVER")].iloc[-2]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = new_df[new_df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    pass
if last_crossover.to_json():
    # slack_message_sender.delay(text=f"Sorted Stock ID {sorted_stock_id}")
    # slack_message_sender.delay(text=f"Last Crossover MACD {sorted_stock.symbol.symbol}    " + str(last_crossover))
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
        # slack_message_sender.delay(text=f"Crossover Signal MACD {sorted_stock.symbol.symbol}    " + str(crossover_signal))
    except:
        pass
df_after_last_crossover

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,macd,macd_signal,macd_diff,percentage,signal
100,M5,219.40,220.30,218.40,220.20,27316,NaN,NaN,2020-02-25 11:20:00,1.499690,1.544125,-0.044435,-0.000666,SELL
101,M5,220.20,221.50,219.50,220.55,69710,NaN,NaN,2020-02-25 11:25:00,1.500457,1.535391,-0.034934,-0.000524,SELL
102,M5,220.55,221.00,220.30,220.60,27601,NaN,NaN,2020-02-25 11:30:00,1.487945,1.525902,-0.037956,-0.000565,SELL
103,M5,220.60,220.75,220.20,220.20,6881,NaN,NaN,2020-02-25 11:35:00,1.429286,1.506579,-0.077292,-0.001105,SELL
104,M5,220.20,220.20,219.00,220.00,24560,NaN,NaN,2020-02-25 11:40:00,1.351090,1.475481,-0.124391,-0.001681,SELL
105,M5,220.05,221.70,219.60,221.50,46772,NaN,NaN,2020-02-25 11:45:00,1.394054,1.459196,-0.065142,-0.000908,SELL
106,M5,221.50,221.50,221.00,221.05,34555,NaN,NaN,2020-02-25 11:50:00,1.375939,1.442544,-0.066605,-0.000916,SELL
107,M5,221.05,221.20,220.05,220.20,13238,NaN,NaN,2020-02-25 11:55:00,1.278275,1.409690,-0.131416,-0.001680,SELL
108,M5,220.20,220.90,219.00,220.05,23970,NaN,NaN,2020-02-25 12:00:00,1.175226,1.362798,-0.187571,-0.002204,SELL
109,M5,220.05,220.65,220.00,220.50,7670,NaN,NaN,2020-02-25 12:05:00,1.116988,1.313636,-0.196648,-0.002197,SELL


In [2]:
# MACD Crossover
from market_analysis.tasks.intraday_functions_strategies import *

In [3]:
# raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [4]:
import requests
from bs4 import BeautifulSoup
user = UserProfile.objects.get(user__email="sonupal129@gmail.com")
payload = {"username": 308687, "password": "Sonupal!@9", "password2fa": 1990, "TOKEN":"fbe2c9352ed86c06c20f003bee0babee7d6831a413026b670a907bf27789a4ff"}
headers= {"Host": "www1.nseindia.com", 
            "Cache-Control": "max-age=0",
            "Upgrade-Insecure-Requests": "1", "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
             "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
             }
# print(response.text.encode('utf8'))
# url_request.text.encode("utf-8")
response = requests.post(user.get_authentication_url(), data=payload)
response.content

b'{"message":"When Content-Type:application/x-www-form-urlencoded, URL cannot include query-string parameters (after \'?\'): \'/index/dialog/authorize?apiKey=VpcuCaUDK91y9JPy358T19nUWtq0UF6J8Yl3qVmr&redirect_uri=http%3A%2F%2F127.0.0.1%3A8000%2F&response_type=code\'"}'

In [5]:
# from django.conf import settings
# def import_daily_losers_gainers():
#     urls = {
#             "BUY": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/niftyGainers1.json",
#                     "https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/jrNiftyGainers1.json"],
#             "SELL": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/niftyLosers1.json",
#                     "https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/jrNiftyLosers1.json"]
#         }
    
#     nifty_movement = Symbol.objects.get(symbol="nifty_50").get_nifty_movement()
    
#     def response_filter(obj):
#         open_price = obj.get("openPrice")
#         change = float(obj.get("netPrice"))
#         if "," in open_price:
#             open_price = open_price.replace(",","")
#         open_price = float(open_price)
#         if open_price >= 100 and open_price <= 300 and change >= 1.2:
#             return obj
        
    
#     if nifty_movement in ("BUY", "SELL"):
#         import_urls = urls.get(nifty_movement)
#         if import_urls:
#             for url in import_urls:
#                 response = requests.get(url, headers=settings.NSE_HEADERS)
#                 if response.status_code == 200:
#                     responses = filter(response_filter, response.json().get("data"))
#                     for symbol in responses:
#                         try:
#                             stock = Symbol.objects.get(symbol=symbol.get("symbol").lower())
#                         except:
#                             stock = None
#                         if stock:
#                             SortedStocksList.objects.get_or_create(symbol=stock, entry_type=nifty_movement, created_at__date=datetime.now().date())
#                     return f"Data imported successfully! from {url}"
#                 return slack_message_sender.delay(channel="#random", text=f"Incorrect Url: {url}")
#             return "All Urls Data Imported Succefully"